## Init For Agents Testing

In [1]:
# Setup: Load environment variables and dependencies
import os
import sys
from pathlib import Path

from jinja2 import Environment, FileSystemLoader

project_root = Path.cwd()
src_path = project_root / "src"

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"✓ Added to sys.path: {src_path}")

from LabAgentSkill import skills_utils
from LabAgentSkill.SkillAwareAgent import SkillAwareAgent

root_dir = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
env_path = root_dir / ".env"
env = {}

if env_path.exists():
    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, value = line.split("=", 1)
        env[key.strip()] = value.strip()

# Set API key
os.environ["OPENAI_API_KEY"] = env.get("OPENAI_API_KEY", os.environ.get("OPENAI_API_KEY", ""))
env = Environment(loader=FileSystemLoader('prompts/'))  
skills_folder = Path("/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills")
all_skills = skills_utils.read_all_skills_metadata(skills_folder)
for skill in all_skills:
    print(f"  - {skill['name']}: {skill['description']}")

✓ Added to sys.path: /home/snt/projects_lujun/LabAgentSkill/src


/home/snt/projects_lujun/LabAgentSkill/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  - IMDB-doc: Extract and summarize key information from IMDB movie reviews, including sentiment, plot points, and character analysis.
  - langgraph-docs: Use this skill for requests related to LangGraph in order to fetch relevant documentation to provide accurate, up-to-date guidance.
  - movie-sentiment-analysis: Analyze text sentiment through concise reasoning steps using contextual understanding and keyword cues.
  - sales-analytics: Database schema and business logic for sales data analysis including customers, orders, and revenue.
  - sentiment-analytics: Analyze text sentiment through concise reasoning steps using contextual understanding and keyword cues.


## Load Data - Sentimental Analysis

In [2]:
from datasets import load_dataset
dataset_name = "Volavion/imdb-sampled-300"
loaded_dataset = load_dataset(dataset_name, split="train")
loaded_df = loaded_dataset.to_pandas()
loaded_df = loaded_df.sample(1)

## Select skills 

In [ ]:
# Initialize agents
agent_skill_aware = SkillAwareAgent(use_chat_history=True, use_trim_messages = True, model = "gpt-4o-mini")
agent_skill_exec_agent = SkillAwareAgent(use_chat_history=True, use_trim_messages = True, model = "gpt-4o-mini")
p_exec_imdb_temp = env.get_template('p_exec_imdb.jinja')
p_skill_select_temp = env.get_template('p_skill_select.jinja')
p_skill_discov_temp = env.get_template('p_skill_discov.jinja')
p_skill_exec_temp = env.get_template('p_skill_exec.jinja')
p_default_system_temp = env.get_template('p_default_system.jinja')
# Store results for evaluation
results = []
skill_count = 0
# Process each sample
for idx, row in loaded_df.iterrows():
    text = row["text"]
    true_label = row["label_name"]

    # Step 1: Skill Selection
    # Build skill context from all available skills
    print (f"Start Skill Selection Phase for Sample {idx + 1}/{len(loaded_df)}")
    skill_context = "\n".join([
        f"- **{skill['name']}**: {skill['description']}"
        for skill in all_skills
    ])
    
    p_skill_select = p_skill_select_temp.render(SKILL_CONTEXT=skill_context)
    p_exec_imdb = p_exec_imdb_temp.render(text=text)
    skill_select_resp = agent_skill_aware.chat(user_input=p_exec_imdb,custom_system_prompt=p_skill_select)
    selected_skills = skills_utils.parse_skills_from_json_response(json_response=skill_select_resp,skills_hub_dir=skills_folder)

    skill_execution_context = ""
    for skill_meta in selected_skills:
        skill_execution_context += (
            f"SKill {skill_count + 1}: \n"
            f"{skill_meta['description']}\n"
            f"{'\n'.join(skill_meta['body'].split('\n')[1:])}\n\n"
        )
        skill_count += 1
    
    skill_count_prev = skill_count

    # Step 2: Skill Discovery - Progressive disclosure of skill context to find more skills inside Skills
    print (f"Number of selected skills: {len(selected_skills)}")
    print (f"Start Skill Discovery Phase for Sample {idx + 1}/{len(loaded_df)}")
    while len(selected_skills) > 0:
        p_skill_discov = p_skill_discov_temp.render(SKILL_CONTEXT=skill_execution_context)
        skill_discov_resp = agent_skill_exec_agent.chat(user_input=p_skill_discov, custom_system_prompt=p_default_system_temp.render())
        selected_skills = skills_utils.parse_skills_from_json_response(json_response=skill_discov_resp,skills_hub_dir=skills_folder)

        for skill_meta in selected_skills:
            skill_execution_context += (
                f"SKill {skill_count + 1}: \n"
                f"{skill_meta['description']}\n"
                f"{'\n'.join(skill_meta['body'].split('\n')[1:])}\n\n"
            )
            skill_count += 1
    print (f"End of skill discovery phase. Found total of new skills: {skill_count - skill_count_prev}")

    # Step 3: Query Execution After Enough Skill Context is Gathered
    p_skill_exec = p_skill_exec_temp.render(SKILL_CONTEXT=skill_execution_context)
    imdb_exec_response = agent_skill_exec_agent.chat(user_input=p_exec_imdb, custom_system_prompt=p_skill_exec)
    message_classification = skills_utils.parse_message_from_json_response(imdb_exec_response)
    is_correct = true_label.lower() in message_classification.strip().lower()
    
    # Store result
    results.append({
        "index": idx,
        "text": text[:100] + "..." if len(text) > 100 else text,
        "true_label": true_label,
        "selected_skills": [s["name"] for s in selected_skills],
        "correct": is_correct
    })
    print (f"Agent Final Decision Messsage: {message_classification}")
    print(f"Correct: {'✓' if is_correct else '✗'}")

# Calculate and display accuracy
accuracy = sum(r["correct"] for r in results) / len(results)
print(f"\n{'='*60}")
print(f"Overall Accuracy: {accuracy:.2%} ({sum(r['correct'] for r in results)}/{len(results)})")
print(f"{'='*60}")

✓ SkillAwareAgent initialized
  Model: gpt-4o-mini
  Chat History: ENABLED ✓
  Trim Messages: ENABLED ✓
✓ SkillAwareAgent initialized
  Model: gpt-4o-mini
  Chat History: ENABLED ✓
  Trim Messages: ENABLED ✓
Start Skill Selection Phase for Sample 211/1
Number of selected skills: 1
Start Skill Discovery Phase for Sample 211/1
End of skill discovery phase. Found total of new skills: 0
Agent Final Decision Messsage: ```json
{
  "Message": "positive — Strong praise for Peter Sellers' performance and a desire to see the movie again, emphasizing its comedic value."
}
```
Correct: ✓

Overall Accuracy: 100.00% (1/1)


In [4]:
agent_skill_exec_agent.display_human_and_ai_message_history()


╔══════════════════════════════════════════════════════════════════════════════╗
║ CONVERSATION HISTORY                                                         ║
║ Total Messages: 4 | Turns: 2                                                 ║
╚══════════════════════════════════════════════════════════════════════════════╝

┌─ TURN 1 ────────────────────────────────────────────────────────────────────────
│
│ 👤 USER:
│   You are a skill selection assistant. You will be given several skill
│   descriptions.
│   
│   Your task is to identify and select any referenced or required additional
│   skills mentioned in the descriptions.
│   
│   For example, if a description says: "You must explicitly read the skill
│   'NETFLIC-doc'", then you should include NETFLIC-doc in your selection.
│   
│   Return only valid JSON in the following format:
│   
│   {
│   "Skills": ["skill_name_1", "skill_name_2"]
│   }
│   
│   SKill 1: 
│   Analyze text sentiment through concise reasoning steps using co

In [6]:
skill_select_resp

'{\n  "Message": "The sentiment of the review is positive. The reviewer expresses admiration for Peter Sellers and the film, indicating a desire to see it again and share it with others, particularly highlighting the positive effect of laughter.",\n  "Skills": ["movie-sentiment-analysis"]\n}'